In [1]:
# PyData
import pandas
import numpy
from pandas import DataFrame, Series
from sklearn import datasets, linear_model, preprocessing, cross_validation
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, Ridge

# System
import datetime
import os
import math

# Graphing
#%matplotlib inline # Only works on Python 3 in the docker container
#import seaborn # Only works on Python 3 in the docker container

#os.environ['PYSPARK_PYTHON'] = 'python2'


%matplotlib inline

# Spark
import pyspark
from pyspark.sql import SQLContext

from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint

pyspark.SparkContext.setSystemProperty('spark.executor.memory', '30g')

sc = pyspark.SparkContext('local[2]')
sqlContext = SQLContext(sc)

In [2]:
train_df = DataFrame.from_csv("train.csv", index_col=['Date', 'Store'], parse_dates=['Date'])[['Customers']]
df_basic = DataFrame.from_csv("train_features_basic.csv", index_col=['Date', 'Store'])
train_df = train_df[['Customers']].join(df_basic, how='outer')
len(train_df)

/opt/conda/lib/python3.4/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


1017209

In [3]:
train_df.head()

Customers  DayOfWeek  Open  Promo  StateHoliday  \
Date       Store                                                    
2015-07-31 1            555          5     1      1             0   
           2            625          5     1      1             0   
           3            821          5     1      1             0   
           4           1498          5     1      1             0   
           5            559          5     1      1             0   

                  SchoolHoliday  cal:dayOfMonth  cal:dayOfWeek  cal:dayofyear  \
Date       Store                                                                
2015-07-31 1                  1              31              4            212   
           2                  1              31              4            212   
           3                  1              31              4            212   
           4                  1              31              4            212   
           5                  1              31              4            212   

                  cal:month  cal:quarter  cal:weekofyear  cal:year  
Date       Store                                                    
2015-07-31 1              7            3              31      2015  
           2              7            3              31      2015  
           3              7            3              31      2015  
           4              7            3              31      2015  
           5              7            3              31      2015

# Create Daily Grouped Mean

In [4]:
train_df.reset_index(inplace=True)

In [5]:
means_df = train_df[train_df['Customers'] > 0]
means_df['Open'] = train_df['Open'].fillna(1)


/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [6]:
def calculate_means(df):
    means = df.groupby([ 'Store', 'DayOfWeek', 'Promo' ])['Customers'].mean()
    means = means.reset_index()
    return means

means_df = calculate_means(means_df)
train_df = pandas.merge(train_df, means_df, on = ['Store','DayOfWeek','Promo'], how='left', suffixes=["", "_predicted_mean"])


In [7]:
### NOTE fillna greatly reduces score. Choose a better fill (per-customer-fill)

train_df['Customers_predicted_mean'] = train_df['Customers_predicted_mean'].fillna(train_df['Customers_predicted_mean'].mean())


In [8]:
def score(predicted, actual): # RootMeanSquaredPercentError RMSPE
    score_df = DataFrame.from_dict({"Predicted": predicted, "Actual": actual})
    score_df = score_df[score_df['Actual'] != 0]
    score_df['PercentError'] =  (score_df['Actual'] - score_df['Predicted']) / score_df['Actual']
    score_df['PercentErrorSquared'] = score_df['PercentError'] * score_df['PercentError']
    
    return math.sqrt(score_df['PercentErrorSquared'].mean())

score(train_df['Customers'], train_df['Customers_predicted_mean'])

0.430820056115827

# Create Spark Random Forest Model

In [9]:
# Shuffle columns

del train_df['Date']

feature_columns = list(train_df.columns)
feature_columns.remove("Customers")
feature_columns = ['Customers'] + feature_columns
train_df = train_df[feature_columns]

In [10]:
df = sqlContext.createDataFrame(train_df)
df = df.map(lambda row: LabeledPoint(row[0], row[1:])).toDF()


In [11]:
%%time
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=1200).fit(df)
(trainingData, testData) = df.randomSplit([0.8, 0.2])

rf = RandomForestRegressor(numTrees=8, maxDepth=10, maxBins=1200, featuresCol="indexedFeatures")

pipeline = Pipeline(stages=[featureIndexer, rf])


CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 26 s


In [12]:
%%time
model = pipeline.fit(trainingData)


CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 1min 46s


In [13]:
# Make predictions.
predictions = model.transform(testData)

# Compute RMPSE
squares = predictions.rdd.filter(lambda x: x.label != 0).map(lambda x: ((x.label - x.prediction) / x.label) *  ((x.label - x.prediction) / x.label))
math.sqrt(squares.mean())

0.13594857758655635

# Featurize Test data

In [14]:

test_df = DataFrame.from_csv("test_features_basic.csv", index_col='Date')
test_df.head()

,Store,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year
Date,,,,,,,,,,,,,,
2015-09-17,1,1,4,1,1,0,0,17,3,260,9,3,38,2015
2015-09-17,3,2,4,1,1,0,0,17,3,260,9,3,38,2015
2015-09-17,7,3,4,1,1,0,0,17,3,260,9,3,38,2015
2015-09-17,8,4,4,1,1,0,0,17,3,260,9,3,38,2015
2015-09-17,9,5,4,1,1,0,0,17,3,260,9,3,38,2015


In [15]:

test_df = pandas.merge(test_df, means_df, on = ['Store','DayOfWeek','Promo'], how='left', suffixes=["", "_predicted_mean"])
test_df['Customers_predicted_mean'] = test_df['Customers'].fillna(test_df['Customers'].mean())
del test_df['Customers']


In [16]:
test_df.head()

,Store,Id,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,cal:dayOfMonth,cal:dayOfWeek,cal:dayofyear,cal:month,cal:quarter,cal:weekofyear,cal:year,Customers_predicted_mean
0,1,1,4,1,1,0,0,17,3,260,9,3,38,2015,571.735294
1,3,2,4,1,1,0,0,17,3,260,9,3,38,2015,833.779412
2,7,3,4,1,1,0,0,17,3,260,9,3,38,2015,987.112676
3,8,4,4,1,1,0,0,17,3,260,9,3,38,2015,853.436620
4,9,5,4,1,1,0,0,17,3,260,9,3,38,2015,590.500000


In [18]:

feature_columns = list(test_df.columns)
feature_columns.remove("Id")
feature_columns = ['Id'] + feature_columns
test_df = test_df[feature_columns]

In [26]:

test_df['Open'] = test_df['Open'].fillna(1)

In [27]:
df = sqlContext.createDataFrame(test_df)
df = df.map(lambda row: LabeledPoint(row[0], row[1:])).toDF()


In [30]:
predictions = model.transform(df).map(lambda row: [row.label, row.prediction]).collect()

In [34]:
predictions_df = DataFrame.from_records(predictions, columns=['Id', 'Prediction'])

In [36]:
predictions_df.set_index("Id", inplace=True)

In [37]:
predictions_df.to_csv("test-features-predicted_customers.csv")